# Advanced Time-Series Forecasting for Economic Indicators

## Multivariate Forecasting with ARIMA, Prophet, and VAR Models

This notebook demonstrates advanced time-series forecasting techniques to predict key Kenyan economic indicators. We will use the cleaned dataset from the data exploration phase to build and compare several powerful models.

### Key Models & Concepts:
- **ARIMA**: A robust baseline model for univariate time-series forecasting.
- **Prophet**: A powerful forecasting tool from Facebook that excels with seasonality and trend decomposition.
- **Vector Autoregression (VAR)**: A multivariate model that captures the linear interdependencies among multiple time series.
- **Model Evaluation**: Rigorous comparison of model performance using standard metrics and visualizations.
- **Forecasting**: Generating future predictions based on the best-performing models.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Time-series modeling libraries
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import adfuller
from prophet import Prophet

# Model evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('talk')

print("📚 Libraries imported successfully")

In [ ]:
# --- Load Cleaned Data ---
# This assumes you have run the Data_Exploration.ipynb notebook first
# to generate the cleaned and merged dataset.
# For simplicity, we will replicate the key data loading and cleaning steps here.

data_dir = '../data/raw/'

def load_and_prepare_data():
    """Loads, cleans, and merges the key economic datasets."""
    # Load GDP
    gdp = pd.read_csv(f'{data_dir}Annual GDP.csv')
    gdp.columns = ['Year', 'Nominal_GDP_Ksh_M', 'Real_GDP_Ksh_M', 'Real_GDP_Growth']
    gdp['Year'] = pd.to_datetime(gdp['Year'], format='%Y')
    gdp.set_index('Year', inplace=True)
    for col in gdp.columns:
        gdp[col] = pd.to_numeric(gdp[col].astype(str).str.replace(',', ''), errors='coerce')

    # Load Debt
    debt = pd.read_csv(f'{data_dir}Public Debt.csv', skiprows=3)
    debt = debt.iloc[:, :2]
    debt.columns = ['Date', 'Total_Debt_Ksh_B']
    debt['Date'] = pd.to_datetime(debt['Date'], errors='coerce')
    debt.dropna(subset=['Date'], inplace=True)
    debt.set_index('Date', inplace=True)
    debt['Total_Debt_Ksh_B'] = pd.to_numeric(debt['Total_Debt_Ksh_B'].astype(str).str.replace(',', ''), errors='coerce')
    debt = debt.resample('A').last()

    # Load FX
    fx = pd.read_csv(f'{data_dir}Monthly exchange rate (end period).csv', skiprows=3)
    fx = fx.iloc[:, :2]
    fx.columns = ['Date', 'KES_USD']
    fx['Date'] = pd.to_datetime(fx['Date'], errors='coerce')
    fx.dropna(subset=['Date'], inplace=True)
    fx.set_index('Date', inplace=True)
    fx['KES_USD'] = pd.to_numeric(fx['KES_USD'].astype(str).str.replace(',', ''), errors='coerce')
    fx = fx.resample('A').mean()

    # Merge and create features
    df = gdp.join(debt, how='inner').join(fx, how='inner')
    df['Debt_to_GDP_Ratio'] = (df['Total_Debt_Ksh_B'] * 1000) / df['Nominal_GDP_Ksh_M']
    df.dropna(inplace=True)
    
    return df

df = load_and_prepare_data()
target_variable = 'Real_GDP_Growth'
exog_variables = ['Total_Debt_Ksh_B', 'KES_USD', 'Debt_to_GDP_Ratio']

print("📊 Loaded and prepared data for modeling.")
print(f"Target Variable: {target_variable}")
print(f"Exogenous Variables: {exog_variables}")
print(df.head())

## 1. Data Generation and Preparation

We'll create synthetic economic data that mimics real-world patterns:
- Long-term trend
- Seasonal patterns
- Economic cycles
- Random noise

In [ ]:
# Split data into training and testing sets
train_size = int(len(df) * 0.8)
train, test = df[0:train_size], df[train_size:len(df)]

print(f"Training set size: {len(train)}")
print(f"Test set size: {len(test)}")

# Fit the ARIMA model
# Note: ARIMA parameters (p,d,q) should be determined through rigorous analysis (e.g., ACF/PACF plots)
# For this example, we use common starting values.
arima_model = ARIMA(train[target_variable], order=(1, 1, 1)).fit()
print("\n📊 ARIMA Model Summary")
print(arima_model.summary())

# Make predictions
arima_preds = arima_model.forecast(steps=len(test))
arima_rmse = np.sqrt(mean_squared_error(test[target_variable], arima_preds))
print(f"\nARIMA Test RMSE: {arima_rmse:.2f}")

In [ ]:
# Plot ARIMA forecast vs actuals
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train[target_variable], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test.index, y=test[target_variable], mode='lines', name='Test Data (Actuals)'))
fig.add_trace(go.Scatter(x=test.index, y=arima_preds, mode='lines', name='ARIMA Forecast', line=dict(dash='dash')))

fig.update_layout(
    title=f'ARIMA Forecast for {target_variable}',
    xaxis_title='Year',
    yaxis_title=target_variable,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.show()

## 3. Advanced Forecasting with Prophet

Prophet is a forecasting tool developed by Facebook that is robust to missing data and shifts in trend, and typically handles seasonality well.

In [ ]:
# Prepare data for Prophet
prophet_train_df = train.reset_index().rename(columns={'Year': 'ds', target_variable: 'y'})

# Initialize and fit the model
prophet_model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
prophet_model.fit(prophet_train_df)

# Create future dataframe
future = prophet_model.make_future_dataframe(periods=len(test), freq='A')
prophet_forecast = prophet_model.predict(future)

# Extract predictions and calculate RMSE
prophet_preds = prophet_forecast['yhat'][-len(test):]
prophet_rmse = np.sqrt(mean_squared_error(test[target_variable], prophet_preds))
print(f"Prophet Test RMSE: {prophet_rmse:.2f}")

## 4. Multivariate Forecasting with Vector Autoregression (VAR)

The VAR model captures the linear interdependencies among multiple time series. It's a powerful tool for multivariate forecasting.

In [ ]:
# Check for stationarity (required for VAR)
def check_stationarity(df):
    """Perform ADF test for stationarity."""
    results = {}
    for col in df.columns:
        adf_test = adfuller(df[col])
        results[col] = {'ADF Statistic': adf_test[0], 'p-value': adf_test[1]}
    return pd.DataFrame(results)

print("Stationarity Check (p-values):")
# We use diff() to make the series stationary if needed
stationarity_results = check_stationarity(train[[target_variable] + exog_variables].diff().dropna())
print(stationarity_results.loc['p-value'])

# Fit the VAR model on differenced data
var_data = train[[target_variable] + exog_variables].diff().dropna()
var_model = VARMAX(var_data, order=(1, 0)).fit(disp=False)
print("\n📊 VAR Model Summary")
print(var_model.summary())

# Forecast
var_forecast = var_model.forecast(steps=len(test))
var_preds = var_forecast[target_variable]

# Invert the differencing
var_preds_inversed = train[target_variable].iloc[-1] + var_preds.cumsum()
var_rmse = np.sqrt(mean_squared_error(test[target_variable], var_preds_inversed))
print(f"\nVAR Test RMSE for {target_variable}: {var_rmse:.2f}")

In [ ]:
# Plot Prophet forecast components
fig = prophet_model.plot_components(prophet_forecast)
plt.suptitle(f'Prophet Model Components for {target_variable}', y=1.02)
plt.show()

In [ ]:
# Plot VAR forecast vs actuals
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train[target_variable], mode='lines', name='Training Data'))
fig.add_trace(go.Scatter(x=test.index, y=test[target_variable], mode='lines', name='Test Data (Actuals)'))
fig.add_trace(go.Scatter(x=test.index, y=var_preds_inversed, mode='lines', name='VAR Forecast', line=dict(dash='dash')))

fig.update_layout(
    title=f'VAR Forecast for {target_variable}',
    xaxis_title='Year',
    yaxis_title=target_variable,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.show()

## 5. Comprehensive Model Comparison

Let's compare the performance of all our models on the test set.

In [ ]:
# Create a comparison dataframe
comparison_df = pd.DataFrame({
    'Actual': test[target_variable],
    'ARIMA': arima_preds,
    'Prophet': prophet_preds.values,
    'VAR': var_preds_inversed.values
}, index=test.index)

# Plot all forecasts
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df[target_variable], mode='lines', name='Historical Data'))
fig.add_trace(go.Scatter(x=comparison_df.index, y=comparison_df['Actual'], mode='lines+markers', name='Actual Test Data'))
fig.add_trace(go.Scatter(x=comparison_df.index, y=comparison_df['ARIMA'], mode='lines', name='ARIMA Forecast', line=dict(dash='dot')))
fig.add_trace(go.Scatter(x=comparison_df.index, y=comparison_df['Prophet'], mode='lines', name='Prophet Forecast', line=dict(dash='dashdot')))
fig.add_trace(go.Scatter(x=comparison_df.index, y=comparison_df['VAR'], mode='lines', name='VAR Forecast', line=dict(dash='dash')))

fig.update_layout(
    title=f'Model Comparison for {target_variable} Forecasting',
    xaxis_title='Year',
    yaxis_title=target_variable,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.show()

In [ ]:
# Display RMSE scores
rmse_scores = {
    'ARIMA': arima_rmse,
    'Prophet': prophet_rmse,
    'VAR': var_rmse
}
rmse_df = pd.DataFrame(rmse_scores, index=['RMSE']).T.sort_values('RMSE')

print("📊 MODEL PERFORMANCE (RMSE)")
print("="*30)
print(rmse_df)

print("\n🎯 CONCLUSION:")
print(f"The best performing model based on RMSE is **{rmse_df.index[0]}**.")
print("This comprehensive analysis demonstrates how different advanced models can be applied to forecast complex economic indicators.")

## 5. Summary and Insights

Key findings from the predictive modeling analysis.